In [1]:
from alpha_vantage.cryptocurrencies import CryptoCurrencies
import pandas_ta as ta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

key = os.environ['av_key']

In [2]:
cc = CryptoCurrencies(key=key, output_format='pandas')

In [3]:
df, meta_data = cc.get_crypto_intraday('ETH', market='USD', interval='1min', outputsize='full')

In [4]:
df = df.rename(columns={"4. close": "close",
                   "1. open": "open",
                   "2. high": "high",
                   "3. low": "low",
                   "5. volume": "volume"})

df = df[['open', 'high', 'low', 'close', 'volume']]

In [5]:
df = df.sort_index()
df['idx_int'] = np.arange(0, len(df))
df = df.reset_index()

In [6]:
df.head()

,date,open,high,low,close,volume,idx_int
0,2024-11-13 08:26:00,3168.45,3171.64,3167.31,3171.33,40.0,0
1,2024-11-13 08:27:00,3171.50,3172.78,3169.17,3172.15,63.0,1
2,2024-11-13 08:28:00,3171.64,3172.28,3167.01,3168.85,199.0,2
3,2024-11-13 08:29:00,3168.72,3171.91,3167.68,3171.14,36.0,3
4,2024-11-13 08:30:00,3171.08,3172.35,3169.78,3170.25,33.0,4


In [7]:
def compute_gradient(start_index, df, x_label, y_label, len_data):
    # Ensure we only take data points from n to n+5
    if start_index + len_data > len(df):
        return None  # Return None if there are not enough points to calculate gradient
    
    # Extract the last 5 data points (x, y) from the DataFrame
    data_segment = df.iloc[start_index:start_index + len_data]
    x = data_segment[x_label]
    y = data_segment[y_label]
    
    # Calculate the necessary summations for the least squares formula
    n = len(x)
    sum_x = x.sum()
    sum_y = y.sum()
    sum_x2 = (x**2).sum()
    sum_xy = (x * y).sum()
    
    # Calculate the slope (gradient) using the least squares formula
    slope = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x**2)
    return slope

def check_crossing(df, col1, col2):
    # Calculate the difference between the two columns
    diff = df[col1] - df[col2]
    diff = diff/np.abs(diff)
    # Check if there is a sign change in the difference
    crossing = ((diff.shift(1) * diff) - 1)/-2
    
    return crossing

In [8]:
# realtime stock trading
df.ta.ema(length=10, append=True)
df.ta.ema(length=50, append=True)
df.ta.rsi(length=14, append=True)
df.ta.bbands(length=20, std=2, append=True)
df.ta.macd(fast=12, slow=26, signal=9, append=True)
df.ta.psar(append=True)
df.ta.adx(append=True)

,ADX_14,DMP_14,DMN_14
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
345,27.690406,24.992386,16.979754
346,27.076117,23.798873,16.168884
347,25.953040,22.328988,17.775866
348,24.910183,21.300356,16.956984


In [9]:
try: df = df.rename(columns={'EMA_60':'EMA_50'})
except: print("skip")

In [10]:
df['psar_flip_dir'] = 0
df.loc[(df['PSARr_0.02_0.2']==1) & (df['PSARl_0.02_0.2'].isnull()==False), 'psar_flip_dir'] = 1
df.loc[(df['PSARr_0.02_0.2']==1) & (df['PSARs_0.02_0.2'].isnull()==False), 'psar_flip_dir'] = -1

In [11]:
df.head()

,date,open,high,low,close,volume,idx_int,EMA_10,EMA_50,RSI_14,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,ADX_14,DMP_14,DMN_14,psar_flip_dir
0,2024-11-13 08:26:00,3168.45,3171.64,3167.31,3171.33,40.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0,NaN,NaN,NaN,0
1,2024-11-13 08:27:00,3171.50,3172.78,3169.17,3172.15,63.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3167.31,NaN,0.04,0,NaN,NaN,NaN,0
2,2024-11-13 08:28:00,3171.64,3172.28,3167.01,3168.85,199.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3172.7800,0.02,1,NaN,NaN,NaN,-1
3,2024-11-13 08:29:00,3168.72,3171.91,3167.68,3171.14,36.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3172.7800,0.02,0,NaN,NaN,NaN,0
4,2024-11-13 08:30:00,3171.08,3172.35,3169.78,3170.25,33.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3172.6646,0.02,0,NaN,NaN,NaN,0


In [12]:
df['r_ema_s_m'] = df['EMA_10'] / df['EMA_50']

In [13]:
len_data = 3

# Loop through the DataFrame, compute gradient for each row (starting point)
for i in range(len(df) - len_data):  # Make sure we have at least 5 points for each calculation
    gradient = compute_gradient(i, df, 'idx_int', 'EMA_10', len_data)
    df.at[i + len_data-1, 'gradient_ema_10'] = gradient  # Store the gradient in the row corresponding to n+4
    gradient = compute_gradient(i, df, 'idx_int', 'RSI_14', len_data)
    df.at[i + len_data-1, 'gradient_rsi_14'] = gradient  # Store the gradient in the row corresponding to n+4
    gradient = compute_gradient(i, df, 'idx_int', 'close', len_data)
    df.at[i + len_data-1, 'gradient_ls'] = gradient
    gradient = compute_gradient(i, df, 'idx_int', 'EMA_50', len_data)
    df.at[i + len_data-1, 'gradient_ema_50'] = gradient  # Store the gradient in the row corresponding to n+4

In [14]:
# mask_crossing = (df['r_ema_s_m']<1.0005)&(df['r_ema_s_m']>0.9995)
# df['flag_ema_crossing'] = 0
# df.loc[mask_crossing, 'flag_ema_crossing'] = 1

In [15]:
df['flag_ema_crossing'] = check_crossing(df, 'EMA_10', 'EMA_50')

In [16]:
df.iloc[-20:]

,date,open,high,low,close,volume,idx_int,EMA_10,EMA_50,RSI_14,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,ADX_14,DMP_14,DMN_14,psar_flip_dir,r_ema_s_m,gradient_ema_10,gradient_rsi_14,gradient_ls,gradient_ema_50,flag_ema_crossing
330,2024-11-13 13:56:00,3227.63,3228.04,3211.27,3213.32,195.0,330,3227.624528,3203.654259,48.231998,3189.359254,3221.3145,3253.269746,1.983988,0.374911,10.016267,-2.233468,12.249735,NaN,3247.763800,0.04,0,43.251910,26.798412,23.081631,0,1.007482,-1.887538,-8.131068,-10.620,0.704612,-0.0
331,2024-11-13 13:57:00,3213.55,3221.58,3211.01,3221.58,328.0,331,3226.525523,3204.357229,53.275385,3193.350648,3222.7050,3252.059352,1.821721,0.480838,8.862612,-2.709699,11.572311,NaN,3246.304048,0.06,0,40.663470,24.746010,21.502273,0,1.006918,-2.138895,-2.757237,-3.270,0.548745,-0.0
332,2024-11-13 13:58:00,3220.93,3222.88,3212.13,3216.83,198.0,332,3224.762701,3204.846358,50.243985,3197.350037,3223.8345,3250.318963,1.643041,0.367762,7.478835,-3.274780,10.753616,NaN,3244.186405,0.06,0,38.402472,23.766785,19.838195,0,1.006214,-1.430914,1.005994,1.755,0.596049,-0.0
333,2024-11-13 13:59:00,3216.76,3226.28,3214.65,3222.67,305.0,333,3224.382210,3205.545324,53.729931,3202.289009,3225.2250,3248.160991,1.422288,0.444302,6.775320,-3.182637,9.957956,NaN,3242.195821,0.06,0,36.673472,24.219034,18.197399,0,1.005876,-1.071657,0.227273,0.545,0.594047,-0.0
334,2024-11-13 14:00:00,3222.81,3223.79,3212.41,3214.75,240.0,334,3222.630899,3205.906292,48.742469,3204.260493,3225.7365,3247.212507,1.331541,0.244215,5.515126,-3.554264,9.069390,NaN,3240.324672,0.06,0,34.750886,22.277413,18.316551,0,1.005217,-1.065901,-0.750758,-1.040,0.529967,-0.0
335,2024-11-13 14:01:00,3214.75,3216.56,3209.80,3214.62,206.0,335,3221.174372,3206.248006,48.662622,3205.725853,3226.1140,3246.502147,1.263945,0.218121,4.454574,-3.691853,8.146427,NaN,3238.565791,0.08,0,32.600999,21.190636,19.306517,0,1.004655,-1.603919,-2.533655,-4.025,0.351341,-0.0
336,2024-11-13 14:02:00,3213.63,3214.13,3201.94,3209.22,239.0,336,3219.000850,3206.364555,45.340072,3205.489455,3226.0575,3246.625545,1.275120,0.090688,3.142123,-4.003443,7.145566,NaN,3236.264528,0.10,0,30.919795,19.289735,23.135193,0,1.003941,-1.815025,-1.701199,-2.765,0.229131,-0.0
337,2024-11-13 14:03:00,3208.54,3217.28,3204.22,3217.27,236.0,337,3218.686150,3206.792219,50.739655,3205.075310,3225.8805,3246.685690,1.289892,0.293068,2.720208,-3.540287,6.260495,NaN,3232.832075,0.10,0,28.941591,19.726804,21.041554,0,1.003709,-1.244111,1.038517,1.325,0.272107,-0.0
338,2024-11-13 14:04:00,3216.57,3220.48,3211.96,3214.87,154.0,338,3217.992304,3207.108995,49.179822,3204.301663,3225.5670,3246.832337,1.318549,0.248487,2.167195,-3.274640,5.441835,NaN,3229.742868,0.10,0,27.051956,20.792756,19.783735,0,1.003393,-0.504273,1.919875,2.825,0.372220,-0.0
339,2024-11-13 14:05:00,3214.97,3225.47,3211.22,3224.12,291.0,339,3219.106431,3207.776093,54.930611,3204.834478,3225.8555,3246.876522,1.303284,0.458720,2.447116,-2.395775,4.842891,NaN,3226.962581,0.10,0,25.889483,22.175478,17.860654,0,1.003532,0.210141,2.095478,3.425,0.491937,-0.0


In [17]:
mask_ema_grad_pos = (df['gradient_ema_10']>0.05)
mask_ema_grad_neg = (df['gradient_ema_10']<-0.05)
df['flag_grad_ema'] = 0
df.loc[mask_ema_grad_pos, 'flag_grad_ema'] = 1
df.loc[mask_ema_grad_neg, 'flag_grad_ema'] = -1

mask_ema_grad_pos = (df['gradient_ema_50']>0.05)
mask_ema_grad_neg = (df['gradient_ema_50']<-0.05)
df['flag_grad_ema_50'] = 0
df.loc[mask_ema_grad_pos, 'flag_grad_ema_50'] = 1
df.loc[mask_ema_grad_neg, 'flag_grad_ema_50'] = -1

mask_rsi_grad_pos = (df['gradient_rsi_14']>=1)
mask_rsi_grad_neg = (df['gradient_rsi_14']<=1)
df['flag_grad_rsi'] = 0
df.loc[mask_rsi_grad_pos, 'flag_grad_rsi'] = 1
df.loc[mask_rsi_grad_neg, 'flag_grad_rsi'] = -1

df['flag_grad_ls'] = 0
df.loc[df['gradient_ls'] >= 0.05, 'flag_grad_ls'] = 1
df.loc[df['gradient_ls'] <= -0.05, 'flag_grad_ls'] = -1

In [18]:
df['ema_short_above_or_below'] = 0
df.loc[(df['EMA_10']>df['EMA_50']), 'ema_short_above_or_below'] = 1
df.loc[(df['EMA_10']<df['EMA_50']), 'ema_short_above_or_below'] = -1

In [19]:
df['r_close_bbu'] = df['close'] / df['BBU_20_2.0']
df['r_close_bbl'] = df['close'] / df['BBL_20_2.0']
df['r_ema_bbu'] = df['EMA_10'] / df['BBU_20_2.0']
df['r_ema_bbl'] = df['EMA_10'] / df['BBL_20_2.0']

# Strategy

To create a real-time trading strategy with specific technical indicator settings and strategies for entering and exiting trades, let's break down the components clearly:

### 1. **Technical Indicators**
Here's how we can set up the specific technical indicators:

#### Moving Averages
- **Short-Term Moving Average (MA)**: Use a 10-period MA. This is effective for spotting short-term trends and suitable for minute-by-minute trading data.
- **Long-Term Moving Average**: Use a 50-period MA to confirm the overall trend direction. This helps filter out the noise and provides a clearer direction.

#### Relative Strength Index (RSI)
- **Length**: A 14-period RSI is standard and works well in a variety of market conditions.
- **Overbought Threshold**: Set at 70 to indicate potential selling points.
- **Oversold Threshold**: Set at 30 to indicate potential buying points.

#### MACD (Moving Average Convergence Divergence)
- **Fast Line**: 12-period Exponential Moving Average (EMA)
- **Slow Line**: 26-period EMA
- **Signal Line**: 9-period EMA of the MACD line itself

### 2. **Entry and Exit Points**
- **Go Long (Buy)**: 
  - **Entry Point**: When the 10-period MA crosses above the 50-period MA and the RSI is below 70 but above 30, indicating neither overbought nor oversold conditions. Additionally, the MACD line should cross above the signal line.
  - **Exit Point (Take Profit)**: Consider exiting when the 10-period MA crosses back below the 50-period MA, or RSI approaches 70, indicating a potential overbought condition.
  
- **Go Short (Sell)**: 
  - **Entry Point**: When the 10-period MA crosses below the 50-period MA and the RSI is above 30 but below 70, indicating normal conditions. Also, the MACD line should cross below the signal line.
  - **Exit Point (Take Profit)**: Consider exiting when the 10-period MA crosses back above the 50-period MA, or RSI approaches 30, indicating a potential oversold condition.

### 3. **Stop Loss Criteria**
- **For Long Positions**: Place a stop loss below the most recent significant low or a fixed percentage, such as 2-3% below the entry price, depending on your risk tolerance.
- **For Short Positions**: Place a stop loss above the most recent significant high or a fixed percentage, such as 2-3% above the entry price.

### 4. **Risk Management**
- Consider the amount of capital at risk per trade, commonly 1-2% of your total trading capital to manage risk effectively.
- Use trailing stops to lock in profits as the market moves in your favor.

### 5. **Automation and Monitoring**
- Automate the strategy using a trading bot if possible, especially to handle the high-frequency nature of minute-by-minute trading.
- Regularly monitor performance and adjust parameters as necessary based on market conditions and strategy performance.

This setup gives you a robust framework for trading on a minute-by-minute basis, leveraging these technical indicators to make informed trading decisions. Adjust these settings based on backtesting results and your specific risk tolerance.

# Buy Signal

In [20]:
mask_le1 = (df['ema_short_above_or_below']==1)&(df['flag_ema_crossing']==1)&(df['flag_grad_ema']>0)
mask_le2 = (df['MACDh_12_26_9']>0)
mask_le3 = (df['r_close_bbl']<=1.0005)
mask_le4 = (df['RSI_14']<70)&(df['RSI_14']>30)
mask_le5 = (df['PSARl_0.02_0.2'] < df['close']) & (df['psar_flip_dir']>0)
mask_le6 = (df['RSI_14']<40)
mask_le7 = (df['flag_grad_ema']>=0)

In [21]:
df['ema_crossing_pos'] = 0
df.loc[mask_le1, 'ema_crossing_pos'] = 1
df['macd_pos'] = 0
df.loc[mask_le2, 'macd_pos'] = 1
df['close_to_bbl'] = 0
df.loc[mask_le3, 'close_to_bbl'] = 1
df['rsi_30_to_70'] = 0
df.loc[mask_le4, 'rsi_30_to_70'] = 1
df['PSAR_bellow_close'] = 0
df.loc[mask_le5, 'PSAR_bellow_close'] = 1

In [22]:
df['buy_signal'] = np.nan
# df.loc[(mask_le1 & mask_le4) | (mask_le5 & mask_le4 & mask_le2) | (mask_le2 & mask_le6 & mask_le3), 'long_entry'] = 1
df.loc[(mask_le1 & mask_le4) | (mask_le6 & mask_le7), 'buy_signal'] = 1

In [23]:
# df['long_entry'] = df[['ema_crossing_pos', 'macd_pos', 'rsi_30_to_70', 'PSAR_bellow_close']].sum(axis=1)

# Sell signal

In [24]:
mask_lex1 = (df['ema_short_above_or_below']==-1)&(df['flag_ema_crossing']==1)
mask_lex2 = (df['RSI_14']>55)
mask_lex3 = (df['psar_flip_dir'] == -1)
mask_lex4 = (df['flag_grad_ema']<0)
mask_lex5 = (df['MACDh_12_26_9']<0)

In [25]:
df['ema_crossing_neg'] = 0
df.loc[mask_lex1, 'ema_crossing_neg'] = 1
df['rsi_above_70'] = 0
df.loc[mask_lex2, 'rsi_above_70'] = 1
df['psar_flip_neg'] = 0
df.loc[mask_lex3, 'psar_flip_neg'] = 1
df['macd_neg'] = 0
df.loc[mask_lex5, 'macd_neg'] = 1

In [26]:
df['sell_signal'] = np.nan
df.loc[(mask_lex1) | (mask_lex2 & mask_lex4), 'sell_signal'] = 1

# Bullish / bearish confirmation

In [27]:
mask_bulber = (df['ADX_14']>=25)
mask_bul = (df['DMP_14']>=25)
mask_ber = (df['DMN_14']>=25)

df['trend_confirm'] = 0
df.loc[mask_bulber & mask_bul, 'trend_confirm'] = 1
df.loc[mask_bulber & mask_ber, 'trend_confirm'] = -1

# Over-sold/buy Marker

In [28]:
mask_os1 = (df['RSI_14']<=20)
mask_os2 = (df['r_close_bbl']<=1.000)
mask_ob1 = (df['RSI_14']>=80)
mask_ob2 = (df['r_close_bbu']>=1.000)

In [29]:
df['oversold_confirm'] = 0
df.loc[mask_os1, 'oversold_confirm'] = 1
df.loc[mask_ob1, 'oversold_confirm'] = -1

# Check data

In [30]:
df.to_clipboard()

In [31]:
df.iloc[-30:]

,date,open,high,low,close,volume,idx_int,EMA_10,EMA_50,RSI_14,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,ADX_14,DMP_14,DMN_14,psar_flip_dir,r_ema_s_m,gradient_ema_10,gradient_rsi_14,gradient_ls,gradient_ema_50,flag_ema_crossing,flag_grad_ema,flag_grad_ema_50,flag_grad_rsi,flag_grad_ls,ema_short_above_or_below,r_close_bbu,r_close_bbl,r_ema_bbu,r_ema_bbl,ema_crossing_pos,macd_pos,close_to_bbl,rsi_30_to_70,PSAR_bellow_close,buy_signal,ema_crossing_neg,rsi_above_70,psar_flip_neg,macd_neg,sell_signal,trend_confirm,oversold_confirm
320,2024-11-13 13:46:00,3218.31,3224.16,3217.90,3224.16,148.0,320,3211.548530,3189.189178,72.306139,3176.606541,3201.2235,3225.840459,1.537972,0.965868,10.866394,2.163925,8.702469,3189.289703,NaN,0.08,0,38.605872,33.216266,12.321135,0,1.007011,2.468388,-0.240910,1.510,1.337939,-0.0,1,1,-1,1,1,0.999479,1.014970,0.995570,1.011000,0,1,0,0,0,NaN,0,1,0,0,NaN,1,0
321,2024-11-13 13:47:00,3224.15,3232.30,3221.02,3231.33,258.0,321,3215.145161,3190.841760,75.319541,3176.926398,3203.5640,3230.201602,1.662998,1.021181,11.811144,2.486941,9.324204,3192.079327,NaN,0.10,0,39.693083,37.029327,11.114844,0,1.007617,3.199590,2.907519,6.490,1.539981,-0.0,1,1,1,1,1,1.000349,1.017125,0.995339,1.012030,0,1,0,0,0,NaN,0,1,0,0,NaN,1,0
322,2024-11-13 13:48:00,3231.34,3240.84,3230.58,3237.44,380.0,322,3219.198768,3192.669142,77.560310,3176.551804,3206.0205,3235.489196,1.838335,1.033100,12.904141,2.863950,10.040191,3196.101394,NaN,0.12,0,41.171696,41.072816,10.142195,0,1.008310,3.825119,2.627086,6.640,1.739982,-0.0,1,1,1,1,1,1.000603,1.019168,0.994965,1.013426,0,1,0,0,0,NaN,0,1,0,0,NaN,1,0
323,2024-11-13 13:49:00,3237.48,3244.52,3236.49,3244.51,295.0,323,3223.800810,3194.702116,79.841045,3176.401637,3208.9100,3241.418363,2.026131,1.047551,14.177410,3.309775,10.867635,3201.470027,NaN,0.14,0,42.719848,41.399464,9.445518,0,1.009108,4.327825,2.260752,6.590,1.930178,-0.0,1,1,1,1,1,1.000954,1.021442,0.994565,1.014922,0,1,0,0,0,NaN,0,1,0,0,NaN,1,0
324,2024-11-13 13:50:00,3244.69,3248.20,3233.49,3245.79,312.0,324,3227.798845,3196.705563,80.232768,3174.767672,3210.9495,3247.131328,2.253653,0.981464,15.115528,3.398314,11.717214,3207.497023,NaN,0.16,0,44.323306,39.444366,8.318282,0,1.009727,4.300038,1.336229,4.175,2.018211,-0.0,1,1,1,1,1,0.999587,1.022371,0.994046,1.016704,0,1,0,0,0,NaN,0,1,0,0,NaN,1,-1
325,2024-11-13 13:51:00,3245.43,3246.84,3228.01,3234.44,502.0,325,3229.006327,3198.185345,67.675063,3174.702796,3212.2655,3249.828204,2.338705,0.795166,14.772851,2.444510,12.328341,3214.009499,NaN,0.16,0,44.737342,33.871860,11.254573,0,1.009637,2.602759,-6.082991,-5.035,1.741614,-0.0,1,1,-1,-1,1,0.995265,1.018817,0.993593,1.017105,0,1,0,1,0,NaN,0,1,0,0,NaN,1,0
326,2024-11-13 13:52:00,3234.74,3235.71,3228.97,3230.09,152.0,326,3229.203359,3199.436508,63.568499,3177.033646,3214.1650,3251.296354,2.310482,0.714441,13.989011,1.328536,12.660475,3219.479980,NaN,0.16,0,45.121805,32.122538,10.673328,0,1.009304,0.702257,-8.332134,-7.850,1.365472,-0.0,1,1,-1,-1,1,0.993478,1.016700,0.993205,1.016421,0,1,0,1,0,NaN,0,1,0,0,NaN,1,0
327,2024-11-13 13:53:00,3230.12,3242.72,3230.08,3237.42,160.0,327,3230.697294,3200.926056,67.182249,3180.147515,3216.6165,3253.085485,2.267537,0.785222,13.800203,0.911782,12.888421,3224.075183,NaN,0.16,0,45.903046,34.326713,9.665205,0,1.009301,0.845483,-0.246407,1.490,1.370356,-0.0,1,1,-1,1,1,0.995184,1.018009,0.993118,1.015895,0,1,0,1,0,NaN,0,1,0,0,NaN,1,0
328,2024-11-13 13:54:00,3237.26,3238.96,3233.07,3234.56,261.0,328,3231.399604,3202.245035,64.494133,3182.167255,3218.4260,3254.684745,2.253197,0.722484,13.266860,0.302752,12.964108,3227.935154,NaN,0.16,0,46.628484,32.773301,9.227818,0,1.009104,1.098123,0.462817,2.235,1.404263,-0.0,1,1,-1,1,1,0.993817,1.016464,0.992846,1.015471,0,1,0,1,0,NaN,0,1,0,0,NaN,1,0
329,2024-11-13 13:55:00,3234.49,3235.19,3226.39,3228.12,179.0,329,3230.803312,3203.259739,58.789859,3185.519149,322

In [32]:
!pip install python-telegram-bot

  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/654.9 kB ? eta -:--:--
   --------------------------------------- 654.9/654.9 kB 26.2 MB/s eta 0:00:00
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
